<a href="https://colab.research.google.com/github/Akomon333/Prediction-of-insurancy-charges/blob/main/InsuranceChargesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
from google.colab import drive
import opendatasets as od
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import sklearn.linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
drive.mount("/content/drive")

In [ ]:
od.download("https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset")

In [ ]:
path = "./medical-insurance-cost-dataset/insurance.csv"
data = pd.read_csv(path)
print(data)

In [32]:
def data_engineering(df):
    fixed_data = df.copy()

    fixed_data["smoker"] = fixed_data["smoker"].map({"yes": 1, "no": 0})
    fixed_data["sex"] = fixed_data["sex"].map({"female": 1, "male": 0})
    fixed_data["region"] = fixed_data["region"].map({
        "southwest": 0, "southeast": 1, "northwest": 2, "northeast": 3
    })

    numeric_cols = fixed_data.select_dtypes(include=["int", "float"]).columns

    poly = PolynomialFeatures(degree=3, include_bias=False)
    X_poly = poly.fit_transform(fixed_data[numeric_cols])

    poly_features = poly.get_feature_names_out(numeric_cols)
    fixed_data_poly = pd.DataFrame(X_poly, columns=poly_features, index=fixed_data.index)

    return fixed_data_poly

In [34]:
engineered_data = data_engineering(data)
X = engineered_data.drop("charges", axis=1).values
y = engineered_data["charges"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.transform(X_train),
                              columns=engineered_data.drop("charges", axis=1).columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test),
                             columns=engineered_data.drop("charges", axis=1).columns)



scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()


model = sklearn.linear_model.LinearRegression()


model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)


model.fit(X_train_scaled, y_train_scaled)

y_pred_scaled = model.predict(X_test_scaled)

y_pred_scaled = model.predict(X_test_scaled)


y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

Mean Absolute Error: 93.87267507049026
